In [2]:
import os

fileList=os.listdir('./data/html/')

# HTML Files with BeautifulSoup

In [3]:
from bs4 import BeautifulSoup
soups=[]
for file in fileList:
    with open(os.path.join('./data/html/',file)) as f:
        
        soup=BeautifulSoup(f.read())
        soups.append(soup)

In [4]:
fileList

['2020 coronavirus pandemic in South Korea - Wikipedia.html',
 '2020 coronavirus pandemic in the United Kingdom - Wikipedia.html',
 '2020 coronavirus pandemic in the United States - Wikipedia.html',
 '2020 coronavirus pandemic in South Africa - Wikipedia.html',
 '2020 coronavirus pandemic in Italy - Wikipedia.html',
 '2020 coronavirus pandemic in Spain - Wikipedia.html',
 '2020 coronavirus pandemic in Iran - Wikipedia.html',
 '2020 coronavirus pandemic in Japan - Wikipedia.html']

How to find relevant text inside <"p"> elements

In [5]:
print((soups[0].find_all('p')[7]).text)

On 26 January, a third case as a 54-year-old South Korean man. He had used a rental car and visited three restaurants, a hotel, a convenience store and met his family before admitting himself to hospital. All these places were disinfected. 



# Spacy for Natural Language Processing

In [6]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [7]:
#all docs for italy soup
docList=[]
for p in soups[4].find_all('p'):
    docList.append(nlp(p.text))

In [8]:
#create single doc for test purpose
doc=docList[3]

Print of all paragraph inside the html, skip if boring

In [9]:
for doc in docList:
    print(doc)





The 2020 coronavirus pandemic in Italy is an ongoing viral pandemic of coronavirus disease 2019 (COVID-19), a novel infectious disease caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2). The first cases of COVID-19 in Italy during the 2019–20 worldwide pandemic were confirmed on 31 January 2020, when two Chinese tourists in Rome tested positive for the virus.[2] One week later an Italian man repatriated back to Italy from the city of Wuhan, China, was hospitalised and confirmed as the third case in Italy.[3] A cluster of cases was later detected, starting with 16 confirmed cases in Lombardy on 21 February,[4] and 60 additional cases and first deaths on 22 February.[5] By the beginning of March, the virus had spread to all regions of Italy.[6]

On 31 January, the Italian government suspended all flights to and from China and declared a state of emergency. In February, eleven municipalities in northern Italy were identified as the centres of the two main Italian c

The Entity-Recognition from Spacy works bad for dates, it does somewhat work for GeoPoliticalEntities:

In [10]:
for token in doc:
    if token.ent_type_ != None:
        print(token,token.ent_type_)

  
New GPE
Zealand GPE
– 

 
On 
4 DATE
March DATE
, 
the ORG
New ORG
Zealand ORG
Ministry ORG
of ORG
Health ORG
confirmed 
its 
second ORDINAL
case 
, 
a 
woman 
in 
her DATE
30s DATE
who 
returned 
to 
Auckland GPE
from 
Northern 
Italy.[567 
] 

 


# Date Matcher for each month

In [11]:
from spacy.matcher import PhraseMatcher
matcher = PhraseMatcher(nlp.vocab)

In [12]:
from spacy.matcher import Matcher

matcher = Matcher(nlp.vocab)

pattern = [ {"IS_DIGIT":True,'OP': '*'},{"TEXT": "January"},{"IS_DIGIT":True,'OP': '?'}]
pattern2 = [ {"IS_DIGIT":True,'OP': '*'},{"TEXT": "February"},{"IS_DIGIT":True,'OP': '?'}]
pattern3 = [ {"IS_DIGIT":True,'OP': '*'},{"TEXT": "March"},{"IS_DIGIT":True,'OP': '?'}]
pattern4 = [ {"IS_DIGIT":True,'OP': '*'},{"TEXT": "December"},{"IS_DIGIT":True,'OP': '?'}]


matcher.add("JAN", None, pattern)
matcher.add("FEB", None, pattern2)
matcher.add("MARCH", None, pattern3)
matcher.add("DEZ", None, pattern4)

In [13]:
matches = matcher(doc)
print("Matches:", [doc[start:end].text for match_id, start, end in matches])

Matches: ['4 March', 'March']


print of all matches, skip if boring

In [14]:
for doc in docList:
    matches = matcher(doc)
    print("Matches:", [doc[start:end].text for match_id, start, end in matches])

Matches: []
Matches: []
Matches: ['31 January', '31 January 2020', 'January', 'January 2020', 'March']
Matches: ['31 January', 'January', 'February', '8 March', '8 March 2020', 'March', 'March 2020', '11 March', '11 March 2020', 'March', 'March 2020']
Matches: ['6 March', '6 March 2020', 'March', 'March 2020', '20 March', 'March']
Matches: ['20 March', '20 March 2020', 'March', 'March 2020', '19 March', 'March']
Matches: ['31 December', '31 December 2019', 'December', 'December 2019', '9 January', '9 January 2020', 'January', 'January 2020', 'January', 'January 2020', 'February', 'February 3']
Matches: []
Matches: ['31 January', 'January', 'January', 'January 23', '28 January', 'January', '31 January', 'January']
Matches: ['6 February', 'February']
Matches: ['22 February', 'February', '26 February', 'February']
Matches: ['14 February', 'February', '16 February', 'February', '19 February', 'February', '21 January', 'January', '20 February', 'February', '19 January', '19 January 2020', '

In [15]:
SentList=[]
for doc in docList:
    matches = matcher(doc)
    
    for match_id, start, end in matches:
        #print(doc[start].sent,'\n')
        if doc[start].sent not in SentList:
            SentList.append(doc[start].sent)
            s=''
            for token in doc[start].sent:
                if token in doc[start:end]:
                    s+="\x1b[31m"+token.text+"\x1b[0m "
                else:
                    s+=token.text+' '
            print(s,'\n')

The first cases of COVID-19 in Italy during the 2019–20 worldwide pandemic were confirmed on 31 January 2020 , when two Chinese tourists in Rome tested positive for the virus.[2 ] One week later an Italian man repatriated back to Italy from the city of Wuhan , China , was hospitalised and confirmed as the third case in Italy.[3 ]  

By the beginning of March , the virus had spread to all regions of Italy.[6 ] 
  

On 31 January , the Italian government suspended all flights to and from China and declared a state of emergency .  

In February , eleven municipalities in northern Italy were identified as the centres of the two main Italian clusters and placed under quarantine .  

On 8 March 2020 , Prime Minister Giuseppe Conte expanded the quarantine to all of Lombardy and 14 other northern provinces , and on the following day to all of Italy , placing more than 60 million people in quarantine.[8][9][10 ] On 11 March 2020 , Conte prohibited nearly all commercial activity except for super

The Italian man visited his wife 's family and had several business meetings in Romania from 18 to 22 February .  

On 28 February , a 45-year - old man from Maramureș who had returned from Italy on 25 February was admitted to the Clinic of Infectious Diseases and then transferred to Cluj.[518 ]  

On 3 March , a 47-year - old man who had travelled in the same plane with the 38-year - old woman was confirmed positive and admitted to the same hospital.[519 ] 
  

  Russia – 
 On 2 March , a Russian citizen who had returned from Italy was diagnosed.[521 ] 
  

  San Marino – 
 On 27 February , San Marino confirmed its first case , an 88-year - old man with pre - existing medical conditions who was hospitalised at Rimini Hospital.[522][523 ] 
  

On 6 March , Serbia registered its first case , a 43-year - old man who had been to Budapest and Italy .  

  Slovakia – 
 An asymptomatic man who had travelled to Venice between 14 and 15 February transmitted the virus to his father and his wife

In [16]:
matchas=[doc[start:end] for match_id,start,end in matcher(doc)]

In [17]:
matchas

[4 March, March]

responseList=[]
for doc in docList:
    matches = matcher(doc)
    d=[]
    matchas=[doc[start:end] for match_id,start,end in matcher(doc)]
    for dol in matchas:
        for token in dol:
            d.append(token)
    
    s=''
    for token in doc:
        
        if token in d:
            s+="\x1b[31m"+token.text+"\x1b[0m "
        else:
            s+=token.text+' '
    print(s+'\n')
    x=input()
    if x=='1':
        responseList.append(doc)
    

In [61]:
matchas

[4 March, March]

In [20]:
len(
responseList)

50

In [22]:
for res in responseList:
    d=[]
    for dol in matchas:
        for token in dol:
            d.append(token)
    s=''
    for token in res:
        if token in d:
                s+="\x1b[31m"+token.text+"\x1b[0m "
        else:
            s+=token.text+' '    
    print(s)
                

On 31 January , the Italian government suspended all flights to and from China and declared a state of emergency . In February , eleven municipalities in northern Italy were identified as the centres of the two main Italian clusters and placed under quarantine . The majority of positive cases in other regions traced back to these two clusters.[7 ] On 8 March 2020 , Prime Minister Giuseppe Conte expanded the quarantine to all of Lombardy and 14 other northern provinces , and on the following day to all of Italy , placing more than 60 million people in quarantine.[8][9][10 ] On 11 March 2020 , Conte prohibited nearly all commercial activity except for supermarkets and pharmacies.[11][12 ] 
 
On 6 March 2020 , the Italian College of Anesthesia , Analgesia , Resuscitation and Intensive Care ( SIAARTI ) published medical ethics recommendations regarding triage protocols that might be employed.[13][14][15 ] By 20 March , Italy had conducted 206,886 tests for the virus.[16 ] 
 
On 31 December

In [43]:
print((soups[4].find_all('span',{'class':"reference-text"})))

[<span class="reference-text">The data on 2020-03-10 does not include cases from the Lombardy region (approx. 600 cases). This brings the geometric average growth for 2020-03-10 and 2020-03-11 to +17%.</span>, <span class="reference-text">The data on 2020-03-11 does not include cases from Abruzzo.</span>, <span class="reference-text">The data on 2020-03-16 does not include cases from Apulia and the autonomous province of Trento (approx. 150 cases).</span>, <span class="reference-text">The data on 2020-03-17 does not include cases from the province of Rimini.</span>, <span class="reference-text">The data on 2020-03-18 does not include cases from Campania and the province of Parma.</span>, <span class="reference-text">Repatriated from abroad.</span>, <span class="reference-text">The cumulative count given in the sources has been updated to reflect the changes due to subsequent corrections.</span>, <span class="reference-text">The latest update from Lombardy – with 47 more cases not inclu

In [40]:
soups[4].span.refernece-text

NameError: name 'text' is not defined

In [56]:
columns=['Country', 'GovernmentResponse','Target','DateComeIntoEffect','DateExpectedToEnd','DateTerminated','Type','Description','Source']

In [34]:
import pandas as pd

In [81]:
responseList[1]

On 6 March 2020, the Italian College of Anesthesia, Analgesia, Resuscitation and Intensive Care (SIAARTI) published medical ethics recommendations regarding triage protocols that might be employed.[13][14][15] By 20 March, Italy had conducted 206,886 tests for the virus.[16]

In [76]:
d=['Italy','prohibited nearly all commercial activity','all of Italy','2020-03-11',None,None,'prohibited',' Conte prohibited nearly all commercial activity except for supermarkets and pharmacies.','https://www.washingtonpost.com/world/europe/merkel-coronavirus-germany/2020/03/11/e276252a-6399-11ea-8a8e-5c5336b32760_story.html']

In [77]:
data.append(d)

In [67]:
data=[['Italy','suspend flights to and from','China','2020-01-31',None,None,'travel ban','Italian government suspended all flights to and from China','https://www.thelocal.it/20200131/italy-suspends-all-china-flights-after-coronavirus-cases-confirmed-in-rome/']]

In [78]:
df = pd.DataFrame(data,columns=columns) 

In [82]:
df

,Country,GovernmentResponse,Target,DateComeIntoEffect,DateExpectedToEnd,DateTerminated,Type,Description,Source
0,Italy,suspend flights to and from,China,2020-01-31,None,None,travel ban,Italian government suspended all flights to an...,https://www.thelocal.it/20200131/italy-suspend...
1,Italy,expanded the quarantine,Lombardy and 14 other northern provinces,2020-03-08,None,None,quarantine,Prime Minister Giuseppe Conte expanded the qua...,https://www.bbc.com/news/world-middle-east-517...
2,Italy,expanded the quarantine,all of Italy,2020-03-09,None,None,lockdown,Prime Minister Giuseppe Conte expanded the qua...,https://www.bbc.co.uk/news/world-europe-51810673
3,Italy,prohibited nearly all commercial activity,all of Italy,2020-03-11,None,None,prohibited,Conte prohibited nearly all commercial activi...,https://www.washingtonpost.com/world/europe/me...


In [83]:
import qgrid


In [89]:
widget=qgrid.show_grid(df,show_toolbar=True)
widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…